In [58]:
import pandas as pd
import os

In [8]:
import requests
from bs4 import BeautifulSoup
import os

#URL of the page containing the link
url = "https://www.countyhealthrankings.org/explore-health-rankings/georgia/data-and-resources"

# Sending a GET request to the webpage
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Finding and downloading Excel files
for link in soup.find_all('a'):
    href = link.get('href')
    if href and href.endswith('.xlsx') and 'County' in href and 'Health' in href and 'Rankings' in href:
        # Sending a GET request to download the Excel file
        full_url = url + href
        file_response = requests.get(full_url)
        
        # Saving the file
        file_napme = os.path.basename(full_url)
        
        # Save the content in a specified directory
        with open(f"./County_Health_Rankings_Data/{file_name}", 'wb') as file:
            file.write(file_response.content)
        
        print(f"Downloaded file: {file_name}")
            


Downloaded file: 2023%20County%20Health%20Rankings%20Georgia%20Data%20-%20v3.xlsx
Downloaded file: 2022%20County%20Health%20Rankings%20Georgia%20Data%20-%20v2.xlsx
Downloaded file: 2021%20County%20Health%20Rankings%20Georgia%20Data%20-%20v1.xlsx
Downloaded file: 2020%20County%20Health%20Rankings%20Georgia%20Data%20-%20v1_0.xlsx


In [59]:
folder_name = "County_Health_Rankings_Data"
filename = "2023 County Health Rankings Georgia Data - v3.xlsx"
full_path = os.path.join(folder_name, filename)

if os.path.exists(full_path):
    print(f"The file {filename} exists and is accessible.")
else:
    print(f"The file {filename} does not exist or is not accessible.")

The file 2023 County Health Rankings Georgia Data - v3.xlsx exists and is accessible.


In [60]:
ranked_measure_data = pd.read_excel(full_path, sheet_name="Ranked Measure Data", header=1)
additional_measure_data = pd.read_excel(full_path, sheet_name="Additional Measure Data", header=1)

In [61]:
print(ranked_measure_data.head())


    FIPS    State    County Unreliable    Deaths  \
0  13000  Georgia       NaN        NaN  143179.0   
1  13001  Georgia   Appling        NaN     379.0   
2  13003  Georgia  Atkinson        NaN     154.0   
3  13005  Georgia     Bacon        NaN     241.0   
4  13007  Georgia     Baker          x       NaN   

   Years of Potential Life Lost Rate  95% CI - Low  95% CI - High   Z-Score  \
0                        8016.730946   7955.977709    8077.484183       NaN   
1                       11113.699673   9345.538818   12881.860528  0.501568   
2                       11429.804405   8738.789750   14120.819061  0.644947   
3                       12649.824162  10392.750491   14906.897832  1.198325   
4                                NaN           NaN            NaN  0.000000   

   YPLL Rate (AIAN)  ...  % Drive Alone (Hispanic) 95% CI - Low  \
0       2896.759045  ...                              65.483151   
1               NaN  ...                                    NaN   
2          

In [62]:
print(additional_measure_data)

      FIPS    State     County  Life Expectancy  95% CI - Low  95% CI - High  \
0    13000  Georgia        NaN        77.309261     77.252649      77.365874   
1    13001  Georgia    Appling        73.872810     72.503273      75.242347   
2    13003  Georgia   Atkinson        73.076566     71.028239      75.124894   
3    13005  Georgia      Bacon        72.459240     70.820697      74.097782   
4    13007  Georgia      Baker        79.801263     76.105578      83.496948   
..     ...      ...        ...              ...           ...            ...   
155  13313  Georgia  Whitfield        77.172717     76.622596      77.722838   
156  13315  Georgia     Wilcox        75.170123     73.133277      77.206968   
157  13317  Georgia     Wilkes        75.214370     73.425028      77.003713   
158  13319  Georgia  Wilkinson        73.225832     71.254196      75.197469   
159  13321  Georgia      Worth        74.163735     72.871869      75.455602   

     Life Expectancy (AIAN)  Life Expec

In [63]:
print(ranked_measure_data.columns.tolist())

['FIPS', 'State', 'County', 'Unreliable', 'Deaths', 'Years of Potential Life Lost Rate', '95% CI - Low', '95% CI - High', 'Z-Score', 'YPLL Rate (AIAN)', 'YPLL Rate (AIAN) 95% CI - Low', 'YPLL Rate (AIAN) 95% CI - High', 'YPLL Rate (AIAN) Unreliable', 'YPLL Rate (Asian)', 'YPLL Rate (Asian) 95% CI - Low', 'YPLL Rate (Asian) 95% CI - High', 'YPLL Rate (Asian) Unreliable', 'YPLL Rate (Black)', 'YPLL Rate (Black) 95% CI - Low', 'YPLL Rate (Black) 95% CI - High', 'YPLL Rate (Black) Unreliable', 'YPLL Rate (Hispanic)', 'YPLL Rate (Hispanic) 95% CI - Low', 'YPLL Rate (Hispanic) 95% CI - High', 'YPLL Rate (Hispanic) Unreliable', 'YPLL Rate (White)', 'YPLL Rate (White) 95% CI - Low', 'YPLL Rate (White) 95% CI - High', 'YPLL Rate (White) Unreliable', '% Fair or Poor Health', '95% CI - Low.1', '95% CI - High.1', 'Z-Score.1', 'Average Number of Physically Unhealthy Days', '95% CI - Low.2', '95% CI - High.2', 'Z-Score.2', 'Average Number of Mentally Unhealthy Days', '95% CI - Low.3', '95% CI - High

In [64]:
# Initialize a dictionary to store the data
data_dict = {}

# Extract correct column names
column_names = ranked_measure_data.columns.to_list()

# Iterate over each row to populate the dictionary
for i, row in ranked_measure_data.iterrows():
    print(row)
    if i < 2:  # Skip the metadata rows
        continue
    
    # Extract the county name
    county = row['County']  # Extract using the correct column name
    if pd.isna(county):  # Skip rows where county name is NaN
        continue
    
    # Initialize a dictionary for the county
    data_dict[county] = {}
    
    # Iterate over each variable and extract the valid average value and its corresponding 95% confidence interval
    variable = None
    for idx in range(3, len(column_names) - 1):  # Start iterating from the 4th column and leave room for CI columns
        col = column_names[idx]
        value = row[col]
        
        # Check if the column is a valid variable with 'Rate' or 'Ratio' and has both 95% CI columns
        if ('Rate' in col or 'Ratio' in col) and ('CI' not in col) and '95% CI' in additional_measure_data_cols[idx + 1] and '95% CI' in additional_measure_data_cols[idx + 2]:
            variable = col
            average_value = value
            ci_low = row[column_names[idx + 1]]
            ci_high = row[column_names[idx + 2]]

            # Check for NaN values in all three attributes before adding to the dictionary
            if (not pd.isna(average_value) and not pd.isna(ci_low) and not pd.isna(ci_high)):
                data_dict[county][variable] = {'average': average_value,
                                            '95% CI - Low': ci_low,
                                            '95% CI - High': ci_high}


FIPS                                 13000
State                              Georgia
County                                 NaN
Unreliable                             NaN
Deaths                            143179.0
                                   ...    
# Workers who Drive Alone          4926936
% Long Commute - Drives Alone         42.2
95% CI - Low.19                  41.770402
95% CI - High.19                 42.629598
Z-Score.33                             NaN
Name: 0, Length: 246, dtype: object
FIPS                                13001
State                             Georgia
County                            Appling
Unreliable                            NaN
Deaths                              379.0
                                   ...   
# Workers who Drive Alone            7300
% Long Commute - Drives Alone        29.1
95% CI - Low.19                  21.41882
95% CI - High.19                 36.78118
Z-Score.33                      -0.790833
Name: 1, Length: 246, dtype: 

In [65]:
# Print the dictionary for a couple of counties to preview the data structure
for county in list(data_dict.keys())[:2]:
    print(county)
    print(data_dict[county])

Atkinson
{'Teen Birth Rate (Black)': {'average': 45.226130653, '95% CI - Low': 26.803871308, '95% CI - High': 71.476784592}, 'Teen Birth Rate (Hispanic)': {'average': 44.709388972, '95% CI - Low': 30.165236992, '95% CI - High': 63.825432779}, 'Teen Birth Rate (White)': {'average': 68.111455108, '95% CI - Low': 52.677394446, '95% CI - High': 86.654471045}, 'Income Ratio': {'average': 4.3559040399, '95% CI - Low': -0.648040692, '95% CI - High': 588}, 'Social Association Rate': {'average': 5.9573454069, '95% CI - Low': 0.8184418166, '95% CI - High': 40.0}, 'Injury Death Rate': {'average': 96.455268869, '95% CI - Low': 68.909058215, '95% CI - High': 131.3446997}}
Bacon
{'YPLL Rate (White)': {'average': 12715.129935, '95% CI - Low': 9992.3862662, '95% CI - High': 15437.873604}, 'Teen Birth Rate (Black)': {'average': 55.813953488, '95% CI - Low': 35.76105315, '95% CI - High': 83.04673859}, 'Teen Birth Rate (Hispanic)': {'average': 58.823529412, '95% CI - Low': 34.86255157, '95% CI - High': 9

In [66]:
additional_measure_data_cols = additional_measure_data.columns.tolist()

for i, row in additional_measure_data.iterrows():
    if i < 2: continue 
    county = row['County']

    if pd.isna(county) or county not in data_dict.keys():
        continue

    for idx in range(3, len(additional_measure_data_cols)-1):  # Use the correct columns list
        col = additional_measure_data_cols[idx]  # Use the correct columns list
        average_value = row[col]
        
        # Ensure we have the '95% CI' columns next
        if ('Rate' in col or 'Ratio' in col) and ('CI' not in col) and '95% CI' in additional_measure_data_cols[idx + 1] and '95% CI' in additional_measure_data_cols[idx + 2]:
            ci_low = row[additional_measure_data_cols[idx + 1]]
            ci_high = row[additional_measure_data_cols[idx + 2]]
            
            # Check that all three values are not NaN
            if not pd.isna(average_value) and not pd.isna(ci_low) and not pd.isna(ci_high):
                data_dict[county][col] = {
                    'average': average_value,
                    '95% CI - Low': ci_low,
                    '95% CI - High': ci_high
                }


In [67]:
import json
with open('data_by_county_2023.json', 'w') as f:
    json.dump(data_dict, f)


In [72]:
import sqlite3
conn = sqlite3.connect('health_rankings.db')
c = conn.cursor()

#Create Table
c.execute('''
CREATE TABLE IF NOT EXISTS Counties (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL UNIQUE
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS Years (
    id INTEGER PRIMARY KEY,
    year INTEGER NOT NULL UNIQUE
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS Metrics (
    id INTEGER PRIMARY KEY,
    county_id INTEGER, 
    year_id INTEGER,
    variable TEXT NOT NULL,
    average REAL,
    ci_low REAL,
    ci_high REAL,
    FOREIGN KEY (county_id) REFERENCES Counties (id),
    FOREIGN KEY (year_id) REFERENCES Years(id)    
)
''')

conn.commit()


In [76]:
import json

# Loading the JSON data
with open('data_by_county_2023.json') as file:
    data = json.load(file)

# Inserting counties and years 
for county in data.keys():
    c.execute("INSERT OR IGNORE INTO Counties (name) VALUES (?)", (county,))
    c.execute("INSERT OR IGNORE INTO Years (year) VALUES (2023)",)

# Inserting metrics
for county, metrics in data.items():
    for variable, values in metrics.items():
        c.execute('''
        INSERT INTO Metrics (county_id, year_id, variable, average, ci_low, ci_high) 
        VALUES ((SELECT id FROM Counties WHERE name = ?),
                (SELECT id FROM Years WHERE year = 2023),
                ?, ?, ?, ?)
        ''', (county, variable, values['average'], values['95% CI - Low'], values['95% CI - High']))

conn.commit()


In [81]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to your database
conn = sqlite3.connect('health_rankings.db')

# Query to fetch data
#First line of query is defining what the column names will appear as in the output
#Join command doesn't actually modify data in database. It temporarily associates (or maps)
# the metrics table ids with their respective values for the purpose of viewing or analyzing 
# in the query result
query = """
SELECT c.name as County, y.year as Year, m.variable as Variable, m.average as Average, m.ci_low as CI_Low, m.ci_high as CI_High
FROM Metrics m
JOIN Counties c ON m.county_id = c.id
JOIN Years y ON m.year_id = y.id
"""

# Load the data into a pandas DataFrame
data = pd.read_sql(query, conn)

# Save the data to a CSV file
data.to_csv('full_database_export.csv', index=False)

# Close the connection
conn.close()
